In [5]:
from nnunetv2.utilities.get_network_from_plans import get_network_from_plans

In [76]:
import torch
import json
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [33]:
# set default CUDA device to GPU 2
dev_id = 2
assert torch.cuda.is_available(), "CUDA is not available."
assert torch.cuda.device_count() > dev_id, f"Requested CUDA device {dev_id} unavailable."
torch.cuda.set_device(dev_id)
print(f"Using {torch.cuda.get_device_name(dev_id)} (cuda:{dev_id})")

Using NVIDIA RTX A6000 (cuda:2)


In [2]:
checkpoint = torch.load("/data/msaqib3/nnUNet_trained_models/Dataset111_7TExVivoFlash/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/checkpoint_best.pth", map_location=torch.device('cpu'), weights_only=False)

In [8]:
plan_info = json.load(open("/data/msaqib3/nnUNet_trained_models/Dataset111_7TExVivoFlash/nnUNetTrainer__nnUNetPlans__3d_fullres/plans.json"))


In [ ]:
info_to_rebuild= plan_info['configurations']['3d_fullres']

In [20]:
plan_info['label_manager']

'LabelManager'

In [26]:
network = get_network_from_plans(
    arch_class_name=info_to_rebuild['architecture']['network_class_name'],
    arch_kwargs=info_to_rebuild['architecture']['arch_kwargs'],
    arch_kwargs_req_import=info_to_rebuild['architecture']['_kw_requires_import'],
    input_channels=1,
    output_channels=10
    )

In [27]:
checkpoint.keys()

dict_keys(['network_weights', 'optimizer_state', 'grad_scaler_state', 'logging', '_best_ema', 'current_epoch', 'init_args', 'trainer_name', 'inference_allowed_mirroring_axes'])

In [28]:
network.load_state_dict(checkpoint['network_weights'])

<All keys matched successfully>

In [57]:
data_file.shape

(1, 96, 96, 96)

In [32]:
encoder = network.encoder

In [62]:
all_patch_files = os.listdir("/data/msaqib3/nnUNet_preprocessed/Dataset111_7TExVivoFlash/nnUNetPlans_3d_fullres/")


In [67]:
all_patch_files = list(filter(lambda x: x.endswith('.npy') and "_e2" in x and "_seg" not in x, all_patch_files))

In [68]:
all_patch_files

['INDD100013L_path_temporal_area_failing_589_217_531_flash_reslice_e2.npy',
 'INDD109198L_edge_effect_518_410_258_flash_reslice_e2.npy',
 'INDD100013L_additional_training_data_290_250_393_flash_reslice_e2.npy',
 'INDD107077L_dots_mf_254_285_996_flash_reslice_e2.npy',
 'INDD112013L_failing_region_358_357_909_flash_reslice_e2.npy',
 'INDD109198L_empty_region_546_388_49_flash_reslice_e2.npy',
 'INDD100013L_visual_461_111_337_flash_reslice_e2.npy',
 'INDD104093R_dots_if_322_392_823_flash_reslice_e2.npy',
 'INDD100013L_additional_training_data_317_282_572_flash_reslice_e2.npy',
 'INDD107077L_dots_mot_192_370_621_flash_reslice_e2.npy',
 'INDD112013L_failing_region_375_341_1022_flash_reslice_e2.npy',
 'INDD100013L_wm_318_193_700_flash_reslice_e2.npy',
 'INDD104093R_dots_mf_565_264_1014_flash_reslice_e2.npy',
 'INDD100013L_additional_training_data_357_266_510_flash_reslice_e2.npy',
 'INDD107077L_dots_orf_596_295_984_flash_reslice_e2.npy',
 'INDD112013L_failing_region_539_294_709_flash_reslice_

In [83]:
for patch_file in tqdm(all_patch_files):
    data_file = np.load(f"/data/msaqib3/nnUNet_preprocessed/Dataset111_7TExVivoFlash/nnUNetPlans_3d_fullres/{patch_file}")
    input_tensor = torch.tensor(data_file).cuda().unsqueeze(0)
    encoder_outputs = encoder.forward(input_tensor)
    embeddings = encoder_outputs[4][0].detach().cpu().numpy()
    np.save(f"/data/msaqib3/nnUNet_embeddings_script/embeddings/{patch_file}", embeddings)

100%|██████████| 327/327 [00:18<00:00, 17.33it/s]


In [88]:
all_embeddings = []
for patch_file in tqdm(all_patch_files):
    embeddings = np.load(f"/data/msaqib3/nnUNet_embeddings_script/embeddings/{patch_file}")
    all_embeddings.append(embeddings.reshape(320,-1))

100%|██████████| 327/327 [00:00<00:00, 719.56it/s]


In [91]:
all_embeddings = np.hstack(all_embeddings)

In [ ]:
all_embeddings =all_embeddings.T

In [97]:
np.save("/data/msaqib3/nnUNet_embeddings_script/all_embeddings.npy", np.array(all_embeddings))

In [ ]:
PCA

In [81]:
torch.load(f"/data/msaqib3/nnUNet_embeddings_script/embeddings/{patch_file.replace('.npy', '_embeddings.pt')}").detach().cpu().numpy()

/tmp/ipykernel_2101767/2751477821.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f"/data/msaqib3/nnUNet_embeddings_script/embeddings/{patch_file.replace('.np

array([[[[ 7.81104788e-02, -1.36828283e-04, -2.84577400e-04,
          -1.23244384e-03, -1.41124614e-03, -6.63569605e-04],
         [ 1.11703046e-01,  1.96915381e-02, -6.46911794e-04,
          -1.00549159e-03, -6.97157288e-04,  3.53231765e-02],
         [ 4.20083627e-02, -6.60203747e-04, -7.07186351e-04,
          -2.98948085e-04, -9.18086211e-04,  1.31796628e-01],
         [ 6.82623163e-02, -8.63079855e-04, -5.32017148e-04,
          -7.33885390e-04, -5.86145790e-04, -5.17111272e-04],
         [ 1.26649633e-01,  2.27540974e-02, -1.57516659e-03,
          -1.12734514e-03, -7.97247747e-04,  3.46325412e-02],
         [ 7.95364082e-02, -4.90747334e-04, -1.21638796e-03,
          -5.53303922e-04, -1.48343621e-04,  4.91745509e-02]],

        [[ 8.50925520e-02,  2.29676813e-02,  7.92804807e-02,
           8.84205103e-02, -2.18223853e-04, -6.81932026e-04],
         [ 2.85433918e-01,  2.89177001e-01,  3.98524523e-01,
           4.72378999e-01,  4.34037507e-01,  2.38869265e-01],
         [ 1.1